# 2.4 Textlicher Teil Bebauungsplan – Embeddings + Vector Store (GPT-4o)

Das Context-Window (128.000 Tokens) in GPT-4o ist begrenzt. Entsprechend muss ab einer gewissen Seitenanzahl des schriftlichen Dokuments eines Bebauungsplans auf Emebddings zurückgegriffen werden. (~765 Tokens pro Bild mit hoher Auflösung / 85 Tokens pro Bild mit niedriger Auflösung)

IDEE: Embeddings dienen ausschließlich dazu die Seitenanzahl des schriftlichen Dokuments zu reduzieren. GPT-4o bekommt anschließend wieder Bilder als Kontext zu sehen. Dadurch gehen möglichst wenig Informationen verloren.

* Zero-Shot
* Chain-Of-Thought

In [64]:
from utils.pdf2prompts import pdf2prompts
import asyncio
import os
from dotenv import load_dotenv, find_dotenv
from dotmap import DotMap
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.schema.messages import AIMessage, HumanMessage
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

load_dotenv(find_dotenv())

model = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    model_name="gpt-4o",
    model_kwargs={"top_p": 0, "seed": 42},
    temperature=0,
)

embeddings_model = OpenAIEmbeddings(
    api_key=os.getenv("OPENAI_API_KEY"),
    model="text-embedding-3-large",
)

In [ ]:
# A) OCR via RapidOCR --> CNN/RNN != Transformer (LLM)
# from langchain_community.document_loaders import PyMuPDFLoader
# pdf_path = "../data/raw/bpläne/2_zeichnung_textteil_getrennt/F 11- 02 Gewerbegebiet Himmelreich - schriftlicher Teil.pdf"
# loader = PyMuPDFLoader(pdf_path, extract_images=True)
# data = loader.load()
# print(data[0])

In [3]:
# B) OCR via GPT-4o / Kompletter schriftlicher Teil
# Idee: Jede Seite einzeln verarbeiten, um den Fokus zu maximieren und Context-Window klein zu halten.
async def page2text(page_prompt):
    msg = model.invoke([
        AIMessage(
            content=[
                {
                    "type": "text",
                    "text": "Du bist ein Assistent um Textinformationen aus dem schriftlichen Teil eines Bebauungsplans zu extrahieren."
                },
            ]
        ),
        HumanMessage(
            content=[
            {
                "type": "text",
                "text": 'Extrahiere den kompletten Textinhalt.'
            },
            page_prompt
            ]
        ),
    ])
    return msg.content

pdf_path = "../data/raw/bpläne/2_zeichnung_textteil_getrennt/F 11- 02 Gewerbegebiet Himmelreich - schriftlicher Teil.pdf"
prompts = pdf2prompts(pdf_path)
prompt_chain = list(map(lambda prompt: page2text(prompt), prompts))
data = await asyncio.gather(*prompt_chain)
print(data[0])

SCHRIFTLICHER TEIL (Teil B)

BEBAUUNGSPLAN "GEWERBEGEBIET HIMMELREICH"

STADT LAICHINGEN, GEMARKUNG FELDSTETTEN, ALB-DONAU-KREIS

Der Geltungsbereich wird durch das Planzeichen im Lageplan begrenzt.

Lageplan M 1: 500

Für die planungsrechtlichen bzw. bauordnungsrechtlichen Festsetzungen gelten:

- Baugesetzbuch (BauGB)
  in der Fassung der Bekanntmachung vom 27.08.1997 (BGBl. I. S. 2141).

- Baunutzungsverordnung (BauNVO)
  in der Fassung der Bekanntmachung vom 23.01.1990 (BGBl. S. 132), zuletzt geändert am 22.04.1993 (BGBl. I. S. 466).

- Planzeichenverordnung 1990 (PlanZV 90)
  in der Fassung der Bekanntmachung vom 18.12.1990 (BGBl. I. S. 58).

- Landesbauordnung (LBO)
  in der Fassung der Bekanntmachung vom 08.08.1995 (GBl. S. 617).

Bisherige Festsetzungen:
Mit in Kraft treten dieses Bebauungsplanes treten im Geltungsbereich alle bisherigen gemeindlichen bauplanungsrechtlichen und bauordnungsrechtlichen Festsetzungen außer Kraft.


In [63]:
# B.1) Mehrere Threads – Extrahierter Text
# Problem: Visueller Kontext geht verloren + fehlender Kontext zwischen Seiten bleibt bestehen = keine gute Idee.
async def summarize(page_content):
    msg = model.invoke([
        AIMessage(
            content=[
                {
                    "type": "text",
                    "text": "Du bist ein Assistent um Textinformationen aus dem schriftlichen Teil eines Bebauungsplans zu extrahieren."
                },
            ]
        ),
        HumanMessage(
            content=[
            {
                "type": "text",
                "text": 'Fasse relevante Informationen zu den folgenden Themen zusammen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche, Dach. Gebe deine Antwort im JSON-Format aus: {<Thema>: <Information>}. Steht keine relevante Information zur Verfügung gebe "-" aus.'
            },
            {
                "type": "text",
                "text": page_content
            },
            ]
        ),
    ])
    return msg.content

prompt_chain = list(map(lambda page_content: summarize(page_content), data))
results = await asyncio.gather(*prompt_chain)
for result in results:
    print(result)

```json
{
  "Art der baulichen Nutzung": "-",
  "Maß der baulichen Nutzung": "-",
  "Bauweise": "-",
  "überbaubare Grundstücksfläche": "-",
  "Dach": "-"
}
```
```json
{
  "Art der baulichen Nutzung": "Eingeschränktes Gewerbegebiet (GEE) und Gewerbegebiet (GE). Zulässig sind Gewerbebetriebe, die das Wohnen nicht wesentlich stören, sowie der Verkauf von auf dem Grundstück produzierten Waren auf einer untergeordneten Fläche. Einzelhandelsbetriebe sind nur mit nicht innenstadtrelevanten Sortimenten zulässig. Vergnügungsstätten und Einzelhandelsbetriebe mit innenstadtrelevanten Sortimenten sind nicht zulässig.",
  "Maß der baulichen Nutzung": "-",
  "Bauweise": "-",
  "überbaubare Grundstücksfläche": "-",
  "Dach": "-"
}
```
```json
{
  "Art der baulichen Nutzung": "-",
  "Maß der baulichen Nutzung": {
    "Grundflächenzahl": "siehe Einträge im Lageplan",
    "Geschossflächenzahl": "siehe Einträge im Lageplan",
    "Höhe der baulichen Anlagen": "siehe Einträge im Lageplan, gemessen von de

In [66]:
# B.2) Create Vector Store
documents = list(map(lambda page: DotMap({"page_content":page, "metadata":{"source": ""}}), data))
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [ ]:
# B.2) Query Vector Store ==> Relevant Pages ()
query = "Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, Dachform und Dachneigung."
retrieved_docs = vectorstore.similarity_search(query, k=3)
print(len(retrieved_docs))

In [ ]:
# B.2) Connect Informations via Chat
# * Add relevant pages to context-window

# TODO retriever/vectorstore -> wie funktioniert das?